# Mecanismo Exponencial

## Importações e definições de variáveis/funções globais

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
from datetime import datetime, date
from dateutil.relativedelta import relativedelta


filename        = "Dataset_Covid_CE.csv"
filename_out    = "Dataset_Covid_CE_Exponential.csv"

epsilons = [0.1, 0.5, 1.0, 10]

# df_municipios = pd.read_csv('ce-cities-lat-lon-format.csv')

## Leitura e limpeza do dataset inicial
Fazemos apenas uma limpeza básica para não modificar muito os resultados do algoritmo.

In [2]:
# definição dos semi-identificadores
semi_ids = [
    "municipioCaso",
    "resultadoFinalExame",
    "racaCor",
]

# definição dos atributos sensíveis
sensitive = [
    "comorbidadeCardiovascularSivep",
    "comorbidadeDiabetesSivep",
]

# definição dos tipos
dtype = {
    "municipioCaso"                  : "str",
    "resultadoFinalExame"            : "str",
    "racaCor"                        : "str",
    "dataNascimento"                 : "str",
    "comorbidadeCardiovascularSivep" : "str",
    "comorbidadeDiabetesSivep"       : "str",
}

# definição das colunas de data
# dates = [
#     "dataNascimento",
# ]
# date_parser = lambda x: pd.to_datetime(x, format="%Y-%m-%d", errors = 'coerce')

# pegar apenas os atributos desejados do dataset
df = pd.read_csv(filename, usecols=semi_ids, dtype=dtype)
df = df.dropna(how="all", subset=semi_ids) # remover registros com todos os semi_ids nulo
# df = df.dropna(how="any", subset=semi_ids) # remover registros com qualquer semi_ids nulo
# df['municipioCaso'] = df['municipioCaso'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') # remove acentuação

df

,municipioCaso,resultadoFinalExame,racaCor
19,SOBRAL,Negativo,Parda
20,PACAJUS,Negativo,Parda
21,HORIZONTE,Negativo,NaN
25,FORTALEZA,Negativo,Parda
28,CAUCAIA,Negativo,Sem Informacao
...,...,...,...
1266272,FORTALEZA,Positivo,Branca
1266273,FORTALEZA,Negativo,Parda
1266274,FORTALEZA,Provável,Branca
1266275,CRATEUS,Negativo,Parda


## Consultas


In [3]:
def exponential(data, universe, score_function, epsilon, verbose=False):
    x = data
    O = universe
    q = score_function
    sensitivity = 1

    # calcula scores para cada valor no universo
    scores = [q(x, o) for o in O]
    
    # calcula probabilidade para cada score
    probabilities = [np.exp((epsilon * score) / (2 * sensitivity)) for score in scores]

    # normaliza probabilidades
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)
    
    if verbose:
        print(universe[:10])
        print(scores[:10])
        print(probabilities[:10])

    # retorna opção aleatória com base nas probabildiades
    return np.random.choice(O, p=probabilities)

### Q1

In [4]:
# municipio com o menor numero de casos positivos de COVID-19
def Q1(df, epsilon, verbose=False):
    table = df.loc[df['resultadoFinalExame'] == 'Positivo', ['municipioCaso', 'resultadoFinalExame']].groupby('municipioCaso').count()
    original_results = table.nsmallest(20, 'resultadoFinalExame')
    universe = df['municipioCaso'].unique()
    x = table.to_dict()['resultadoFinalExame']

    def score(x, o):
        v = x[o] if o in x else 0
        return -1 * ((1 + v)/40)

    if verbose:
        print(f"original_results: {original_results}")

    return exponential(x, universe, score, epsilon, verbose)

# Q1(df, epsilon=0.1, verbose=True)
# Q1(df, epsilon=0.5, verbose=True)
Q1(df, epsilon=1, verbose=True)
# Q1(df, epsilon=10, verbose=True)

dfq1 = pd.DataFrame({epsilon:pd.Series([Q1(df, epsilon=epsilon) for _ in range(20)]) for epsilon in epsilons})
dfq1

original_results:                        resultadoFinalExame
municipioCaso                             
TARRAFAS                                50
PENAFORTE                               58
ARNEIROZ                                59
ANTONINA DO NORTE                       64
AIUABA                                  71
JATI                                    81
MULUNGU                                 86
GENERAL SAMPAIO                        113
BAIXIO                                 114
IBARETAMA                              122
POTENGI                                127
PALHANO                                132
ABAIARA                                141
GUARAMIRANGA                           141
POTIRETAMA                             146
FORTIM                                 162
UMARI                                  179
GRANJEIRO                              183
SAO JOAO DO JAGUARIBE                  183
ARATUBA                                189
['SOBRAL' 'PACAJUS' 'HORIZONTE' 'FOR

,0.1,0.5,1.0,10.0
0,REDENCAO,CARIDADE,SABOEIRO,TARRAFAS
1,PEDRA BRANCA,APUIARES,TARRAFAS,ARNEIROZ
2,CATARINA,GUAIÚBA,ABAIARA,TARRAFAS
3,MARTINOPOLE,GUARAMIRANGA,FORTIM,PENAFORTE
4,INDEPENDENCIA,TARRAFAS,ANTONINA DO NORTE,ANTONINA DO NORTE
5,PIRES FERREIRA,CARIDADE,PENAFORTE,PENAFORTE
6,MISSAO VELHA,ITAPIUNA,PORANGA,ARNEIROZ
7,SAO JOAO DO JAGUARIBE,ANTONINA DO NORTE,PENAFORTE,ARNEIROZ
8,MISSAO VELHA,ALTANEIRA,ANTONINA DO NORTE,TARRAFAS
9,IPAUMIRIM,CATARINA,GENERAL SAMPAIO,AIUABA


In [5]:
dfq1m = [dfq1[col].value_counts().idxmax() for col in dfq1.columns]
dfq1m

['MISSAO VELHA', 'CARIDADE', 'ANTONINA DO NORTE', 'TARRAFAS']

### Q2

In [6]:
# raça com o maior numero de casos negativos de COVID-19
def Q2(df, epsilon, verbose=False):
    table = df.loc[df['resultadoFinalExame'] == 'Negativo', ['racaCor', 'resultadoFinalExame']].groupby('racaCor').count()
    original_results = table.nlargest(20, 'resultadoFinalExame')
    universe = df['racaCor'].dropna().unique()
    x = table.to_dict()['resultadoFinalExame']

    def score(x, o):
        v = x[o] if o in x else 0
        return (1 + v)/200000

    if verbose:
        print(f"original_results: {original_results}")

    return exponential(x, universe, score, epsilon, verbose)

Q2(df, 0.1, verbose=True)
# Q2(df, 10, verbose=True)
# Q2(df, 1, verbose=True)
# Q2(df, 10, verbose=True)

dfq2 = pd.DataFrame({epsilon:pd.Series([Q2(df, epsilon=epsilon) for _ in range(20)]) for epsilon in epsilons})
dfq2

original_results:                 resultadoFinalExame
racaCor                            
Parda                        281535
Branca                        96177
Amarela                       71936
Sem Informacao                34555
Ignorado                      16130
Preta                         10927
Indígena                       2068
['Parda' 'Sem Informacao' 'Amarela' 'Branca' 'Preta' 'Ignorado' 'Indígena']
[1.40768, 0.17278, 0.359685, 0.48089, 0.05464, 0.080655, 0.010345]
[0.1504507  0.1414421  0.14277011 0.14363796 0.14060906 0.14079208
 0.14029799]


,0.1,0.5,1.0,10.0
0,Preta,Branca,Parda,Parda
1,Amarela,Indígena,Amarela,Parda
2,Preta,Branca,Amarela,Parda
3,Sem Informacao,Amarela,Ignorado,Parda
4,Amarela,Amarela,Parda,Parda
5,Indígena,Indígena,Amarela,Parda
6,Indígena,Sem Informacao,Sem Informacao,Parda
7,Indígena,Parda,Branca,Parda
8,Indígena,Branca,Indígena,Parda
9,Ignorado,Ignorado,Sem Informacao,Parda


In [7]:
dfq2m = [dfq2[col].value_counts().idxmax() for col in dfq2.columns]
dfq2m

['Amarela', 'Sem Informacao', 'Amarela', 'Parda']

## Gerar CSVs

In [8]:
dfq1q2 = pd.DataFrame(columns=epsilons, index=['q1', 'q2'], data=[dfq1m, dfq2m])
dfq1q2

,0.1,0.5,1.0,10.0
q1,MISSAO VELHA,CARIDADE,ANTONINA DO NORTE,TARRAFAS
q2,Amarela,Sem Informacao,Amarela,Parda


In [9]:
df.to_csv(filename[:-4] + '_Clean.csv', index=False)
dfq1.to_csv(filename_out[:-4] + '_Q1.csv')
dfq2.to_csv(filename_out[:-4] + '_Q2.csv')
dfq1q2.to_csv(filename_out[:-4] + '_Q1_Q2.csv')